In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm

In [2]:
dataset_path = "/home/ns/robosuite/collects/1705874644_525442/demo101_jan21_image_group.hdf5" 
dataset_path = "/home/ns/collect_robomimic_demos/Elbert/NutAssemblySquare_01_26_2024_06_52PM_elbert/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/Elbert/PickPlaceCan_01_26_2024_06_40PM_Elbert/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/Brooke/PickPlaceCan_01_26_2024_02_19PM_Brooke/demo_image.hdf5"
dataset_path = "/home/ns/collect_robomimic_demos/Lift_01_30_2024_05_03PM_sojib/lift_mixed_v0.hdf5"

# dataset_path = "/home/ns/collect_robomimic_demos/sojib_can_expert/PickPlaceCan_01_30_2024_02_33PM_sojib/demo_image.hdf5"
# dataset_path = "/home/ns/collect_robomimic_demos/sojib_square_expert/NutAssemblySquare_02_01_2024_06_38PM_sojib/square_mix_v0.hdf5"
# dataset_path = "/home/ns/collect_robomimic_demos/sojib_can_expert/PickPlaceCan_01_30_2024_01_45PM_sojib/can_mix_v0.hdf5"

In [3]:
f = h5py.File(dataset_path, "r")
demos = list(f["data"].keys())

In [4]:
lengths=[]
demos_minmax={}
for demo_name in demos:
    demo=f['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

    action=f['data'][demo_name]['actions']
    action=np.array(action) 
    demos_minmax[demo_name] = (np.min(action, axis=0), np.max(action, axis=0))


lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))
print('Median length: ', np.median(lengths))

Number of demos:  300
Max length:  800
Min length:  116
Mean length:  375.99
Median length:  212.0


In [5]:
for key in f['mask'].keys():
    print(key, f['mask'][key])

expert <HDF5 dataset "expert": shape (100,), type "|S16">
failed <HDF5 dataset "failed": shape (100,), type "|S16">
good <HDF5 dataset "good": shape (100,), type "|S16">


In [6]:
f.keys()

<KeysViewHDF5 ['data', 'mask', 'maxs', 'mins']>

In [7]:
np.set_printoptions(precision=4, suppress=True)

In [8]:
mins,maxs=[],[]
for demo_name in demos_minmax.keys():
    min,max= demos_minmax[demo_name]
    mins.append(min)
    maxs.append(max)

mins=np.min(mins, axis=0)
maxs=np.max(maxs, axis=0) 
print('mins: ', mins)
print('maxs: ', maxs)

mins:  [-1.     -1.     -1.1062 -1.     -1.     -1.     -1.    ]
maxs:  [1.2542 1.     1.0814 0.     0.     1.     1.    ]


In [9]:
is_normalized='mins' in f.keys()
print(f'is_normalized: {is_normalized}')

if is_normalized:
    print('mins: ', np.array(f['mins']))
    print('maxs: ', np.array(f['maxs']))

is_normalized: True
mins:  [-0.3861 -0.3678 -1.1138  0.      0.     -0.3    -1.    ]
maxs:  [0.6259 0.3797 1.1079 0.     0.     0.3    1.    ]


In [10]:
f.close()

In [14]:
names=[b.decode('utf-8') for b in f['mask']['expert']]
len(names)

10

In [15]:
names

['demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru',
 'demo-Tru']